# Generate power consumption measure

In this notebook we will simulate a set of edge station timeseries.

In [1]:
import csv
import glob
import json
import overpy
import uuid

from geopy.geocoders import Nominatim

In [3]:
def get_timeseries(fname):
    series = {}
    with open(fname) as f:
        rd = csv.DictReader(f)
        for r in rd:
            series.setdefault(r['dataid'], []).append(
                (r['localminute'], sum(float(r[k])
                                      for k in r if k not in ['localminute', 'dataid'])))
    return series

In [4]:
series = {}
for fname in glob.glob('DumpDB/*.csv'):
    print(fname)
    delta = get_timeseries(fname)
    series.update(delta)

DumpDB/77_101.csv
DumpDB/370_483_545.csv
DumpDB/1114_1169_1185.csv
DumpDB/774_781_890.csv
DumpDB/1192_1283_1310.csv
DumpDB/946_974_1103.csv
DumpDB/114_115.csv
DumpDB/171_187_252.csv
DumpDB/585_661_668.csv


In [5]:
city_name = "cagliari"

# Geocoding request via Nominatim
geolocator = Nominatim(user_agent="city_compare")
geo_results = geolocator.geocode(city_name, exactly_one=False, limit=3)

# Searching for relation in result set
for r in geo_results:
    print(r.address, r.raw.get("osm_type"))
    if r.raw.get("osm_type") == "relation":
        city = r
        break

# Calculating area id
area_id = int(city.raw.get("osm_id")) + 3600000000

# Excecuting overpass call
api = overpy.Overpass()
result = api.query("""
    area(%s)->.searchArea;
    (
      node["amenity"="restaurant"](area.searchArea);
      way["amenity"="restaurant"](area.searchArea);
      relation["amenity"="pub"](area.searchArea);
    );
    out body;
    """ % area_id)

# Printing no. of pubs in nodes and ways
print("Amenities in nodes: %d" % len(result.nodes))
print("Amenities in ways: %d" % len(result.ways))

Casteddu/Cagliari, CA, SAR, 09124, Italia node
Cagliari - Casteddu, CA, SAR, Italia relation
Amenities in nodes: 125
Amenities in ways: 4


In [6]:
n = result.nodes[0]

In [7]:
n.tags

{'addr:city': 'Cagliari',
 'addr:street': 'Bastione di Saint Remy',
 'amenity': 'restaurant',
 'cuisine': 'regional',
 'name': 'Cafe Degli Spiriti',
 'note': 'Sardinian specialties, salads, pizza, ham etc. At night cocktails and loungebar. Great view over the city and the ocean.',
 'phone': '+39 070.311.03.73',
 'price': '€7 drinks'}

In [8]:
def build_station(n):
    code = str(uuid.uuid4())
    station = {'code': code, 'osmid': n.id, 
               'geometry': {'type': "Point", 
                            "coordinates": [float(n.lon), 
                                            float(n.lat)]}, 
               'tags': n.tags}
    return station

In [9]:
def build_node(station):
    code = str(uuid.uuid4())
    return {'code': code, 'stationcode': station['code']}

In [10]:
def build_sensor_type():
    return {
        "code": str(uuid.uuid4()),
        "type": "HomeEnergyMonitorRealPower",
        "name": "temperature sensor in DHT11",
        "brandName": "Acme",
        "modelName": "Acme multisensor DHT11",
        "manufacturerName": "Acme Inc.",
        "category": ["sensor"],
        "function": ["sensing"],
        "controlledProperty": ["realPower"]
        }

In [11]:
def build_sensor(n, node, sensortype):
    return {"code": str(uuid.uuid4()),
            "stypecode": sensortype["code"],
            "nodecode": node['code'],
            "geometry": {"type": "Point", 
                         "coordinates": [float(n.lon), 
                                         float(n.lat)]}
           }

In [12]:
def build_measures(sensor, series):
    return [{"time": t, "measure": {"value": v},
             "sensorcode": sensor['code']} for t, v in series]

In [13]:
series['114'][:5]

[('2017-01-01 00:00:00-06', 1.644),
 ('2017-01-01 00:01:00-06', 1.626),
 ('2017-01-01 00:02:00-06', 1.6199999999999999),
 ('2017-01-01 00:03:00-06', 1.611),
 ('2017-01-01 00:04:00-06', 1.644)]

In [ ]:
stations = []
nodes = []
sensor_types = [build_sensor_type()]
sensors = []
measures = []

for n, k in zip(result.nodes, series):
    station = build_station(n)
    node = build_node(station)
    sensor = build_sensor(n, node, sensor_types[0])    
    stations.append(station)
    nodes.append(node)
    sensors.append(sensor)
    # one measures file per sensors (single file could be huge)
    measures = build_measures(sensor, series[k])
    fn = f'measures_{sensor["code"]}.json'
    print(f'saving {fn}')
    with open(fn, 'w') as f:
        json.dump({'measures': measures}, f, indent=4)

In [15]:
with open('stations.json', 'w') as f:
    json.dump({'stations': stations}, f, indent=4)

In [16]:
with open('nodes.json', 'w') as f:
    json.dump({'nodes': nodes}, f, indent=4)

In [17]:
with open('sensor_types.json', 'w') as f:
    json.dump({'sensor_types': sensor_types}, f, indent=4)

In [18]:
with open('sensors.json', 'w') as f:
    json.dump({'sensors': sensors}, f, indent=4)